

# 1. Data Collection & Analysis

## 1.1. Introduction

Welcome to the Tutorial on Deep Learning for Human Activity Recognition! This is the first notebook in which we will guide you through the  first steps on getting a basic understanding of applying Deep Learning on Human Activity Recognition.

We assume that you already listened to the introduction for this part and that you are familiar with the ultimate goal of the tutorial as well as necessary terms in order to understand this tutorial. If not, you can catch up with the introduction slides on the [tutorial website](https://mariusbock.github.io/dl-for-har).

During this online tutorial we will cover the Deep Learning Activity Recognition Chain (**DL-ARC**) pipeline, which is an overhaul of the Activity Recognition Chain as proposed by [Bulling et al. in 2014](http://dx.doi.org/10.1145/2499621) [[1]](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/intro_and_data_analysis.ipynb?hl=en#scrollTo=t0akNjYypv_y). To familiarise you with the DL-ARC, we will cover within the next hours the following:
- [Data Collection & Analysis](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/data_collection.ipynb?hl=en)
- [Data Preprocessing](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/preprocessing.ipynb?hl=en)
- [Evaluation](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/evaluation.ipynb?hl=en)
- [Training](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/training.ipynb?hl=en)
- [Validation & Testing](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/validation_and_testing.ipynb?hl=en)

We will work with a mix of presentation slides and Juypter Notebooks to implement a basic CNN-LSTM architecture that has proven both interesting and powerful [[2]](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/intro_and_data_analysis.ipynb?hl=en#scrollTo=t0akNjYypv_y). Before jumping into the contents of the pipeline, we will briefly cover the basics of how to use Jupyter Notebooks and, if used, [Google Colab](https://colab.research.google.com/) in the next sections. 

### 1.1.1. For Colab Users

If you are accessing this tutorial via [Google Colab](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/training.ipynb), first make sure to use Google Colab in English. This will help us to better assist you with issues that might arise during the tutorial. There are two ways to change the default language if it isn't English already:
1. On Google Colab, go to `Help` -> `View in English` 
2. Change the default language of your browser to `English`.

To also ease the communication when communicating errors, enable line numbers within the settings of Colab.

1. On Google Colab, go to `Tools` -> `Settings` -> `Editor` -> `Show line numbers`

In general, we strongly advise you to use Google Colab as it provides you with a working Python distribution as well as free GPU resources. To make Colab use GPUs, you need to change the current notebooks runtime type via:

- `Runtime` -> `Change runtime type` -> `Dropdown` -> `GPU` -> `Save`

**Hint:** you can auto-complete code in Colab via `ctrl` + `spacebar`

For the live tutorial, we require all participants to use Colab. If you decide to rerun the tutorial at later points and rather want to have it run locally on your machine, feel free to clone our [GitHub repository](https://github.com/mariusbock/dl-for-har).

To get started with this notebook, you need to first run the code cell below. Please set `use_colab` to be `True` if you are accessing this notebook via Colab. If not, please set it to `False`. This code cell will make sure that imports from our GitHub repository will work.

### 1.1.2. Jupyter Notebooks Basics

[Jupyter notebooks](https://jupyter.org/) are made of two main components. Markdown text cells and code cells. The latter can be seen as small Python scripts, which can be individually run. The output of the code is printed right after the cell. This allows for more granualar and expressive code with explanations and intermediate outputs along the way. In the following you will find a sample code cell. You can run the cell by either clicking the 'run' symbol in the top left of the cell or by clicking on it and hitting `Shift + Enter` on your keyboard. You can also rerun cells as many times as you want, but be aware that some cells require other cells to be run beforehand in order to work properly (e.g. if one cell references variables defined in another cell)

**Note**: If you get "Warning: This notebook was not authored by Google.", just hit "Run anyway".

In [ ]:
# This is a normal print statement
print('hello world!')

In [ ]:
# Just like any script you can declare variables and import packages
import numpy as np

test_array = np.array([1, 2, 3, 4, 5])
print(test_array)

# You can also just write the variable, it will be printed as output of the cell
test_array[0:5:2]

Try for instance to change the last line above to: `test_array[-1]`, to display only the last element, or add some the values to test_array. Also interesting for later tasks is selecting every other element of the array with: `test_array[0:5:2]`.

## 1.2. The Dataset

A critical element in the research field of Activity Recognition is the choice of sensor data, which in turn will influence the type of benchmark dataset we can use. Research is constantly contributing with new datasets, especially as the field of Activity Recognition is expanding and merging with other fields that aim at building  systems that recognize their users' Affect or Attention [[3]](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/intro_and_data_analysis.ipynb?hl=en#scrollTo=t0akNjYypv_y). We will focus in this tutorial on wrist-worn inertial sensors, buildung up from the large amount of commercial devices (such as smartwatches) with such sensors and form factor.

Throughout the whole tutorial we will use the [RealWorld HAR dataset](https://doi.org/10.1109/PERCOM.2016.7456521) [[4]](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/intro_and_data_analysis.ipynb?hl=en#scrollTo=t0akNjYypv_y). The dataset has recorded data from 8 activites (climbing stairs up and down, jumping, lying, running/ jogging, sitting, standing, walking), performed by 15 people. The orginal dataset covers acceleration, GPS, gyroscope, light, magnetic field, and sound level data. Sensors were placed on multiple body positions, i.e. chest, forearm, head, shin, thigh, upper arm, and waist. Each subject performed each activity roughly 10 minutes except for jumping (~1.7 minutes).

To keep things simple and fast, we will use only data obtained from the first three subjects and only use acceleration data captured from the wrist. Within the first task of this notebook, we will show you how to load the dataset using [pandas](https://pandas.pydata.org/) and print the first rows of the dataset.

To get started with this notebook, you need to first run the code cell below. Please set `use_colab` to be `True` if you are accessing this notebook via Colab. If not, please set it to `False`. This code cell will make sure that imports from our GitHub repository will work.

In [ ]:
import os, sys

use_colab = True

module_path = os.path.abspath(os.path.join('..'))

if use_colab:
    # move to content directory and remove directory for a clean start 
    %cd /content/         
    %rm -rf dl-for-har
    # clone package repository (will throw error if already cloned)
    !git clone https://github.com/mariusbock/dl-for-har.git
    # navigate to dl-for-har directory
    %cd dl-for-har/       
else:
    os.chdir(module_path)
    
# this statement is needed so that we can use the methods of the DL-ARC pipeline
if module_path not in sys.path:
    sys.path.append(module_path)

If you expand the `Files` tab (click the folder icon on the left), you should now be able to see the dataset as the directory `dl-for-har`. We are now ready to load the data.

### Task: Loading the dataset

1. Load the dataset containing the data of the first three subjects within the RealWorld HAR dataset, **'rwhar_3sbjs_data.csv'** within the **'data'** folder of the repository, using [pandas](https://pandas.pydata.org/)'  [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method
2. While reading in the data, use the **names** attribute to pass along the header of the CSV file as a list of strings; The column names we will be using are: `'subject_id', 'acc_x', 'acc_y', 'acc_z', 'activity_label'`
3. Print the first rows of the loaded dataset using the [`head()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) method and inspect the data of these first rows

In [ ]:
import pandas as pd

# first declare a variable that holds the dataset's path and filename in one string:


# use pd.read_csv() to load that dataset; use the names attribute to pass along the column names:


# print the first 7 rows of the loaded data using the .head():



As you can see, the dataset consists of 5 columns.
- **subject_id**: identifier which person the data belongs to
- **acc_x**: acceleration data obtained from the wrist (x-axis)
- **acc_y**: acceleration data obtained from the wrist (y-axis)
- **acc_z**: acceleration data obtained from the wrist (z-axis)
- **activity_label**: name of the activity that was performed

## 1.3. Visualizing the Dataset

Once the dataset has been loaded successfully, we now go over some usual steps you can take to quickly inspect a particular dataset and check what its data tend to look like.


We first have a look at the labeling of the dataset. Each record represents a recorded value of the sensor worn by the participants on their wrist. The corresponding label is the activity they were performing at a given point in time. The next code cell will introduce built-in functions of [pandas](https://pandas.pydata.org/), which you can use to get an overview on how the labels are distributed across all records.

### Task: Analyse the labeling
1. Analyze the label distribution of the dataset: What unique labels exist in the dataset? How many instances of each label are there? Use pandas' [`unique()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.unique.html) and [`value_counts()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.value_counts.html) functions
2. Visualize your results obtained above, using a bar plot diagram in [matplotlib](https://matplotlib.org/)

In [ ]:
import matplotlib.pyplot as plt

# first obtain the unique labels within the 'activity_label' column
#  - use pandas' unique() function and print out the resulting array  


# obtain the label distribution of the 'activity_label' column
#   - use pandas' value_counts() function and print out the resulting array


# plot the label counts over all activities, using x and y axes:
# x_axis = the different labels within the dataset
# y_axis = their occurences across the dataset
def plot_stats(x_axis, y_axis):
  plt.figure(figsize=(12, 5))
  plt.bar(x_axis, y_axis, width=0.5)
  plt.xlabel('Activity label')
  plt.ylabel('Count')
  plt.title('Label Distribution')
  plt.show()

# use the above function to display the activity label counts:
#  the x_axis parameter should hold the label strings ('running', etc.), 
#  the y_axis parameter should hold their counts 



Now let us focus on visualizing acceleration data in the next task. Even though raw accelerometer readings are less intelligible than, for instance, images, we can still visualize the accelerometer timeseries along the x axis. Within the next coding task, you will use a function `plot_activity`, which will plot data belonging to sample activities as a simple timeseries plot. Accelerometer data measures both acceleration in three-dimensional space and orientation, leading often to problems in classification when the sensor is worn in different orientations or body placements between datasets [[6, 7]](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/intro_and_data_analysis.ipynb?hl=en#scrollTo=t0akNjYypv_y). Interesting parameters for accelerometers are their sampling rate (in Herz, for instance 100Hz), their range (in g, for instance -8g to 8g). Other factors such as the sensitivty, internal filters used, temperature calibration or internal offsets can for the purpose of human activity recognition be neglected in most MEMS accelerometers.

### Task: Plot the activity data

We use the [plotly](https://plotly.com/python/) library this time to visualize the accelerometer data:
1. Filter the original dataset to only contain records with the wanted label, for instance `'walking'` 
2. Define the y-axis of the plot as the acceleration data on the x-, y- and z-axis (see above when loading the dataset)
3. Define the x-axis of the plot as the time in seconds (using the known sampling rate)
4. Use the given `plot_activity` function for exploring different activity labels, sensors, and other  parameters: 
   - What interesting parts in the dataset can you spot? Did all persons wear the sensors similarly?
   - Compare plots between data that has the original 50 Hz data, with downsampled data: How much data can we leave out before the timeseries start look differently?

In [ ]:
import plotly.graph_objects as go
import numpy as np

# create three variables that hold the activity label, the list of sensor names, 
# and the sampling rate (50):


# this function can be used as is, see at the end of this cell
def plot_activity(data, label, sensor_names, sampling_rate):
    # filter the data to retain only activity_label data
    filtered_data = data[data.activity_label==label]
    
    # define the y- and x-axis as defined above
    # count for x how many records there are, divide by sampling rate
    y_axis = filtered_data[sensor_names]
    x_axis = np.array(range(len(filtered_data))) / sampling_rate

    # plot data
    fig = go.Figure()
    for s in sensor_names: 
      fig.add_trace(go.Scatter(x=x_axis, y=y_axis[s], name=s))
    fig.update_layout(margin_l=0,margin_r=0)  # no margins left or right
    fig.update_layout(title="activity: "+label)
    fig.update_layout(xaxis_title="time (seconds)")
    fig.update_layout(yaxis_title="acceleration (10g)")
    fig.update_layout(legend_x=0, legend_bgcolor='rgba(0,0,0,0.2)')
    fig.show()

# call the above function, using the correct inputs:



## 1.4. More detailled analysis

We finally introduce a more complex analysis of the dataset. As the next notebook will show you, we need to segment our data using a sliding window approach. The size of the windows is often a crucial parameter, which you need to decide upon before training your network. Sometimes, smaller windows are required due to activities changing fast -- smaller windows will also give you essentially more data to train, as you are chunking the data into smaller pieces. On the other hand, small windows can also lead to your network not recognizing characteristical traits within the data and thus hurt the expressiveness of your data. [Muharari and Ploetz](https://dl.acm.org/doi/abs/10.1145/3267242.3267287) phrase it best: "Decisions regarding any sliding window procedure are hard and often final decisions that impact the recognition procedure as a whole. As such mistakes made here are critical and errors made are difficult to recover from." [[5]](https://colab.research.google.com/github/mariusbock/dl-for-har/blob/main/tutorial_notebooks/intro_and_data_analysis.ipynb?hl=en#scrollTo=t0akNjYypv_y)

We therefore implemented [helpful functions](https://github.com/mariusbock/dl-for-har/blob/main/data_processing/data_analysis.py) within our GitHub, which allows you to get a summary of the activities in your dataset. For each subject as well as overall the function `analyze_window_lengths` gives you:
1. A list of all the activities that are included and how long they lasted
2. The average, maximum and minimum time each activity lasted

This can help you better understand your use case at hand, and decide on how large or small your sliding windows should be.

### Task: Analyze the activities

1. Run the code cell below
2. Analyze the results: What do you notice? Do the activities change frequently?
3. Do the results differ a lot across subjects?

In [ ]:
from data_processing.data_analysis import analyze_window_lengths

analyze_window_lengths(labels=data['activity_label'], subject_idx=data['subject_id'], sampling_rate=50)

# References

1.   A. Bulling, U. Blanke, and B. Schiele. 2014. A tutorial on human activity recognition using body-worn inertial sensors. ACM Comput. Surv. 46, 3, Article 33 (January 2014), 33 pages. DOI:https://doi.org/10.1145/2499621
2.   T. Sztyler and H. Stuckenschmidt, "On-body localization of wearable devices: An investigation of position-aware activity recognition," 2016 IEEE International Conference on Pervasive Computing and Communications (PerCom), 2016, pp. 1-9, DOI:https://doi.org/10.1109/PERCOM.2016.7456521.
3.   K. Van  Laerhoven. The Three A’s of Wearable and Ubiquitous Computing: Activity, Affect, and Attention. Frontiers in Computer Science, Section Mobile and Ubiquitous Computing, 3: 57, 2021. DOI:http://doi.org/10.3389/fcomp.2021.691622
4.   Marius Bock, Alexander Hoelzemann, Michael Moeller, and Kristof Van Laerhoven. 2021. Improving Deep Learning for HAR with Shallow LSTMs. In Proceedings of the 2021 International Symposium on Wearable Computers, ISWC 2020, September 21-26, 2021, ACM. DOI:https://doi.org/10.1145/3460421.3480419 
5.   V. S. Murahari and T. Ploetz. 2018. On attention models for human activity recognition. In Proceedings of the 2018 ACM International Symposium on Wearable Computers (ISWC '18). Association for Computing Machinery, New York, NY, USA, 100–103. DOI:https://doi.org/10.1145/3267242.3267287
6.   S. Thiemjarus, "A Device-Orientation Independent Method for Activity Recognition," 2010 International Conference on Body Sensor Networks, 2010, pp. 19-23, DOI:https://doi.org/10.1109/BSN.2010.55.
7.   K. Kunze and P. Lukowicz. 2008. Dealing with sensor displacement in motion-based onbody activity recognition systems. In Proceedings of the 10th international conference on Ubiquitous computing (UbiComp '08). Association for Computing Machinery, New York, NY, USA, 20–29. DOI:https://doi.org/10.1145/1409635.1409639